In [127]:
import struct
from typing import Annotated, Dict, List
import numpy as np
import time
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.neighbors import KNeighborsClassifier
from dataclasses import dataclass

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import random
from sklearn.utils import shuffle

AVG_OVERX_ROWS = 10


In [128]:
TYPE_INDEX='LSH'

In [129]:
class BinaryFile:
    def __init__(self, filename):
        self.filename = filename
        self.vec_size = 70
        self.float_size = 4
        self.int_size = 4

    def insert_row(self, row_id, row_data):
        with open(self.filename, 'ab') as file:
            # Pack the ID and the float values into a binary format
            packed_data = struct.pack(f'i{self.vec_size}f', row_id, *row_data)
            # Write the packed data to the file
            file.write(packed_data)

    def read_row(self, row_id):
        with open(self.filename, 'rb') as file:
            # Calculate the position of the row
            # Size of one row (ID + vec_size * floats)
            position = row_id * \
                (self.int_size + self.vec_size * self.float_size)
            # Seek to the position of the row
            file.seek(position)
            # Read the row
            # Size of one row (ID + vec_size * floats)
            packed_data = file.read(
                self.int_size + self.vec_size * self.float_size)
            data = struct.unpack(f'i{self.vec_size}f', packed_data)
            return np.array(data)

    def insert_records(self, rows: List[Dict[int, Annotated[List[float], 70]]]):
        first_position = None
        last_position = None
        with open(self.filename, 'ab') as file:
            # record the position before writing
            first_position = file.tell()
            for row in rows:
                id, embed = row["id"], row["embed"]
                # Pack the ID and the float values into a binary format
                packed_data = struct.pack(f'i{self.vec_size}f', id, *embed)
                # Write the packed data to the file
                file.write(packed_data)
            # Record the position after writing
            last_position = file.tell()
        # Return the first and last position
        return first_position, last_position

    # read all rows
    def read_all(self):
        rows = []
        with open(self.filename, 'rb') as file:
            # iterate over all rows
            while True:
                # Read the row
                packed_data = file.read(
                    self.int_size + self.vec_size * self.float_size)
                if packed_data == b'':
                    break
                data = struct.unpack(f'i{self.vec_size}f', packed_data)
                rows.append(data)
        return np.array(rows)

    def read_positions_in_range(self, first_position, last_position):
        records = []
        with open(self.filename, 'rb') as file:
            file.seek(first_position)
            while file.tell() < last_position:
                packed_data = file.read(
                    self.int_size + self.vec_size * self.float_size)
                if packed_data == b'':
                    break
                data = struct.unpack(f'i{self.vec_size}f', packed_data)
                records.append(data)
        return np.array(records)

    def insert_position(self, row_id, position):
        with open(self.filename, 'ab') as file:
            packed_data = struct.pack('iii', row_id, *position)
            file.write(packed_data)

    def read_position(self, row_id):
        with open(self.filename, 'rb') as file:
            position = row_id * (self.int_size * 2 + self.int_size)
            file.seek(position)
            packed_data = file.read(self.int_size * 3)
            data = struct.unpack('iii', packed_data)
            return np.array(data)

    def insert_positions(self, rows: List[Dict[int, List[int]]]):
        with open(self.filename, 'ab') as file:
            for row in rows:
                id, position = row["id"], row["position"]
                packed_data = struct.pack('iii', id, *position)
                file.write(packed_data)

    def read_all_positions(self):
        positions = []
        with open(self.filename, 'rb') as file:
            while True:
                packed_data = file.read(self.int_size * 3)
                if packed_data == b'':
                    break
                data = struct.unpack('iii', packed_data)
                positions.append(data)
        return np.array(positions)

In [130]:
# insertion if saved_db is not created

class CreateDatabase:
    def __init__(self, file_path="saved_db.bin", new_db=True) -> None:
        self.file_path = file_path
        # binary file handler
        self.bfh = BinaryFile(self.file_path)
        if new_db:
            # just open new file to delete the old one
            with open(self.file_path, "w") as fout:
                # if you need to add any head to the file
                pass

    def insert_records(self, rows: List[Dict[int, Annotated[List[float], 70]]]):
        self.bfh.insert_records(rows)


db = CreateDatabase(file_path="saved_db.bin")
records_np = np.random.random((1000000, 70))
records_dict = [{"id": i, "embed": list(row)}
                for i, row in enumerate(records_np)]
db.insert_records(records_dict)

In [132]:
class IvfDb:
    def __init__(self, file_path="saved_db.bin") -> None:
        self.file_path = file_path
        # number of clusters
        self.n_clusters1 = 1000
        # self.n_clusters2 = 10
        # binary file handler
        self.bfh = BinaryFile(self.file_path)
        ######################
        # LSH config
        self.dim = 70
        self.num_hashes = 10


    def rertive_embeddings(self):
        # return all rows
        return self.bfh.read_all()[:, 1:]


    #############################################################
    ############     search with cos similarity     #############
    #############################################################
    def _search_with_cos_similarity(self, position_file, cluster_file, scores_id_array, query, top_in_region_num, top_results_num):
      bfh_c_pos = BinaryFile(position_file)
      bfh_c = BinaryFile(cluster_file)
      top_results = []
      for score in scores_id_array:
          # read position of this cluster index (centroid index)
          first_position, second_position = bfh_c_pos.read_position(int(score[1]))[1:]
          # read all vectors in this cluster as [[], [], [], ...]
          region_vectors = bfh_c.read_positions_in_range(first_position, second_position)
          region_vectors_scores = []
          for vec in region_vectors:
              # read id and features of this vector
              id = vec[0]
              embed = vec[1:]
              vector_score = self._cal_score(query, embed)
              region_vectors_scores.append((vector_score, id))

          # get k (top_in_region_num) the nearest vectors of that region
          region_vectors_scores = sorted(region_vectors_scores, reverse=True)[:top_in_region_num]
          # concat to get all results of all regions
          top_results = top_results + region_vectors_scores

      # take the best k (top_results_num) vectors in those vectors
      top_results = sorted(top_results, reverse=True)[:top_results_num]

      # the top_results here has scores and ids sorted on scores
      return top_results
    

    #############################################################
    ##################     search with knn     ##################
    #############################################################
    def _search_with_knn(self, position_file, cluster_file, scores_id_array, query, top_results_num):
      
      all_regions_vec = []
      classes = []

      vec_id_dict = dict()

      bfh_c_pos = BinaryFile(position_file)
      bfh_c = BinaryFile(cluster_file)

      # collect all vectors in all regions to train the knn on
      for score in scores_id_array:
          # read position of this cluster index (centroid index)
          first_position, second_position = bfh_c_pos.read_position(int(score[1]))[1:]
          # read all vectors in this cluster as [[], [], [], ...]
          region_vectors = bfh_c.read_positions_in_range(first_position, second_position)
          for vec in region_vectors:
              # save all vectors and their ids to retrive them back after training the model
              vec_id_dict.update({tuple(vec[1:]): vec[0]})
              # save all embeddings to train the model on
              all_regions_vec.append(vec[1:])

          # save the classes of all vectors to train the knn on
          classes = classes + [score[1] for _ in range(len(region_vectors))]

      # train the knn model with number of neighbors = 10
      knn = KNeighborsClassifier(n_neighbors=10)
      knn.fit(all_regions_vec, classes)

      # this have the ids of nearest vectors to the query
      predictions = knn.kneighbors(query, return_distance=False)

      top_results = []
      for vec in predictions[0]:
          vector_score = self._cal_score(query, all_regions_vec[vec])
          # get the id of this vector from the dict we saved before
          top_results.append((vector_score, vec_id_dict.get(tuple(all_regions_vec[vec]))))
      top_results = sorted(top_results, reverse=True)[:top_results_num]

      # the top_results here has scores and ids sorted on scores
      return top_results


    #############################################################
    #############     our rock star retrive     #################
    #############################################################
    def retrive(self, query: Annotated[List[float], 70], top_k=5):
        if TYPE_INDEX == 'LSH':
            # Flatten the query and compute its hash
            query = query.flatten()
            hash_value = self.hash(query)

            # read all hash key from file
            bfh_hash_table_pos = BinaryFile('positions_hash_table.bin')
            hash_keys = bfh_hash_table_pos.read_all_positions()
            # bfh for the hashtable
            bfh_hash_table = BinaryFile('hash_table.bin')
            
            # Generate all possible hashes
            hashes = self.generate_hashes(hash_value)

            # Initialize an empty list to store the scores
            region_vectors_scores = []

            # For each hash, retrieve the vectors and compute their scores
            for hash_value in hashes:
                if not(hash_value in hash_keys):
                    continue

                # getindex of hash_value in hash_keys
                index = np.where(hash_keys[:,0] == hash_value)[0][0]
                # Get the position of this hash in the file
                first_position, second_position = bfh_hash_table_pos.read_position(int(index))[1:]

                # Read all vectors in this hash
                region_vectors = bfh_hash_table.read_positions_in_range(first_position, second_position)

                # For each vector, compute its score and add it to the list
                for vec in region_vectors:
                    id = vec[0]
                    embed = vec[1:]
                    vector_score = self._cal_score(query, embed)
                    region_vectors_scores.append((vector_score, id))

            # Sort the scores and return the top_k IDs
            region_vectors_scores = sorted(region_vectors_scores, reverse=True)[:top_k]
            return [score[1] for score in region_vectors_scores]
        else:
            scores = []
            centroids_level2 = BinaryFile('centroids_1.bin').read_all()
            for centroid in centroids_level2:
                score_centroid = self._cal_score(query, centroid[1:])
                id = centroid[0]
                scores.append((score_centroid, id))
            scores = sorted(scores, reverse=True)[:30]  

            top_results_level_1 = self._search_with_cos_similarity('positions_cluster_1.bin', 'cluster_1.bin', scores, query, 30, top_k)

            # top_results_level_1 = self._search_with_knn('positions_cluster_1.bin', 'cluster_1.bin', scores, query, top_k)

            # here we assume that if two rows have the same score, return the lowest ID
            return [score[1] for score in top_results_level_1]


    #############################################################
    ####################     clc score     ######################
    #############################################################
    def _cal_score(self, vec1, vec2):
        dot_product = np.dot(vec1, vec2)
        norm_vec1 = np.linalg.norm(vec1)
        norm_vec2 = np.linalg.norm(vec2)
        cosine_similarity = dot_product / (norm_vec1 * norm_vec2)
        return cosine_similarity



    #############################################################
    ###########     save clusters and positions     #############
    #############################################################
    def write_to_file(self, cluster_file_name, position_file_name ,centroids_file_name, clusters, centroids):
        # save all cluster in a file
        # output 2 file cluster, position
        # insert clusters
        # insert each cluster in a file
        open(cluster_file_name, 'w').close()
        open(position_file_name, 'w').close()
        open(centroids_file_name, 'w').close()

        bfh_c = BinaryFile(cluster_file_name)
        bfh_c_pos = BinaryFile(position_file_name)
        bfh_cen = BinaryFile(centroids_file_name)

        # insert clusters and positions
        for cluster_index, cluster_vectors in enumerate(clusters):
            cluster_dict = [{"id": int(row[0]), "embed": row[1:]} for row in cluster_vectors]
            first_position, last_position = bfh_c.insert_records(cluster_dict)
            bfh_c_pos.insert_position(cluster_index, [first_position, last_position])

        # insert centroids
        centroids_dict = [{"id": i, "embed": row} for i, row in enumerate(centroids)]
        bfh_cen.insert_records(centroids_dict)

        for cluster_index, cluster_vectors in enumerate(clusters):
          print(f"Cluster {cluster_index} has {len(cluster_vectors)} vectors.")


    #############################################################
    #############      partial train kmeans    ##################
    #############################################################
    def partial_predict(self, embeds, n_clusters):

        # training_set = embeds[np.random.randint(len(embeds), size=100000 if len(embeds) > 100000 else 1000)]
        training_set = shuffle(embeds)[:1000]

        kmeans = KMeans(n_clusters=n_clusters)
        kmeans.fit([tuple(embed) for embed in training_set])

        # predict rest of data
        cluster_labels = []
        for i, embed in enumerate(embeds):
            cluster_id = kmeans.predict([tuple(embed)])
            cluster_labels.append(cluster_id)

        # centroids which are list of vectors (70 float each)
        centroids = kmeans.cluster_centers_.tolist()

        return centroids, cluster_labels
    

    #############################################################
    #################      training kmeans    ###################
    #############################################################
    def kmeans_training(self, embeds, n_clusters):

        # kmeans = MiniBatchKMeans(
        #     n_clusters=self.n_clusters1, batch_size=1000, random_state=42, n_init=10)
        kmeans = KMeans(n_clusters=n_clusters)
        kmeans.fit([tuple(embed) for embed in embeds])

        # get the labels id of each cluster list of size db each vector to its cluster
        cluster_labels = kmeans.labels_
        centroids = kmeans.cluster_centers_.tolist()

        return centroids, cluster_labels
    
    #############################################################
    ###############      training lsh forest    #################
    #############################################################
    '''
    lsh it generates random vectors and make hash to insert in buckets
    '''
    def lsh_training(self, rows):
        self.random_vectors = np.random.randn(self.dim, self.num_hashes)
        hash_table = {}
        for row in rows:
            hash_value = self.hash(row[1:])
            if hash_value in hash_table:
                hash_table[hash_value].append(row)
            else:
                hash_table[hash_value] = [row]

        return hash_table
    
    def hash(self, input_vector):
        
        # Normalize the input vector and the random vectors
        norm_input_vector = input_vector / np.linalg.norm(input_vector)
        norm_random_vectors = self.random_vectors / np.linalg.norm(self.random_vectors, axis=0)

        # Compute the cosine similarity
        cos_sim = np.dot(norm_input_vector, norm_random_vectors)

        # Determine if the cosine similarity is greater than 0
        bools = (cos_sim > 0).astype('int')

        # Convert the boolean array to a binary string and then to an integer
        return int(''.join(bools.astype('str')), 2)

    def hash2(self, input_vector):
        bools = (np.dot(input_vector, self.random_vectors) > 0).astype('int')
        return int(''.join(bools.astype('str')), 2)
    
    #  write lsh hash table in file with binaryfilehandler
    def write_lsh_hash_table(self, hash_table, file_name, position_file_name):
        open(file_name, 'w').close()
        open(position_file_name, 'w').close()

        bfh = BinaryFile(file_name)
        bfh_pos = BinaryFile(position_file_name)

        for hash_value, rows in hash_table.items():
            hash_dict = [{"id": int(row[0]), "embed": row[1:]} for row in rows]
            first_position, last_position = bfh.insert_records(hash_dict)
            bfh_pos.insert_position(hash_value, [first_position, last_position])

    def generate_hashes(self, hash_value):
        # Convert the hash value to a binary string
        binary_hash = format(hash_value, 'b')

        # Generate all hashes by flipping one bit
        one_bit_flips = [int(binary_hash[:i] + str(1 - int(binary_hash[i])) + binary_hash[i+1:], 2) for i in range(len(binary_hash))]

        # Generate all hashes by flipping two bits
        two_bit_flips = [int(binary_hash[:i] + str(1 - int(binary_hash[i])) + binary_hash[j+1:j] + str(1 - int(binary_hash[j])) + binary_hash[j+1:], 2) for i in range(len(binary_hash)) for j in range(i+1, len(binary_hash))]

        # Return the original hash, one bit flips, and two bit flips
        return [hash_value] + one_bit_flips + two_bit_flips
        
    #############################################################
    ########     second rock star building the index     ########
    #############################################################
    def build_index(self):

        # read all rows
        rows = self.bfh.read_all()

        if TYPE_INDEX == 'LSH':
            ###################### level 1 ######################  

            hash_table = self.lsh_training(rows)
            self.write_lsh_hash_table(hash_table, 'hash_table.bin', 'positions_hash_table.bin')

        else:
            ###################### level 1 ######################  

            # centroids, cluster_labels = self.kmeans_training(rows[:, 1:], self.n_clusters1)
            centroids, cluster_labels = self.partial_predict(rows[:, 1:], self.n_clusters1)

            clusters = [[] for _ in range(self.n_clusters1)]
            
            for i, label in enumerate(cluster_labels):
                clusters[int(label)].append(rows[i])

            self.write_to_file('cluster_1.bin', 'positions_cluster_1.bin', 'centroids_1.bin', clusters, centroids)


            ###################### level 2 ######################  

            # centroids_level2, cluster_labels = self.kmeans_training(centroids, self.n_clusters2)

            # clusters = [[] for _ in range(self.n_clusters2)]

            # for i, label in enumerate(cluster_labels):
            #     clusters[int(label)].append([i]+ list(centroids[i]))

            # self.write_to_file('cluster_2.bin', 'positions_cluster_2.bin', 'centroids_2.bin', clusters, centroids_level2)

In [133]:

@dataclass
class Result:
    run_time: float
    top_k: int
    db_ids: List[int]
    actual_ids: List[int]


def run_queries(db, np_rows, top_k, num_runs):
    results = []
    for _ in range(num_runs):
        query = np.random.random((1, 70))

        tic = time.time()
        db_ids = db.retrive(query, top_k)
        toc = time.time()
        run_time = toc - tic

        tic = time.time()
        actual_ids = np.argsort(np_rows.dot(query.T).T / (np.linalg.norm(
            np_rows, axis=1) * np.linalg.norm(query)), axis=1).squeeze().tolist()[::-1]
        toc = time.time()
        np_run_time = toc - tic

        results.append(Result(run_time, top_k, db_ids, actual_ids))
    return results


def eval(results: List[Result]):
    # scores are negative. So getting 0 is the best score.
    scores = []
    run_time = []
    for res in results:
        run_time.append(res.run_time)
        # case for retireving number not equal to top_k, socre will be the lowest
        if len(set(res.db_ids)) != res.top_k or len(res.db_ids) != res.top_k:
            print('not equal length')
            scores.append(-1 * len(res.actual_ids) * res.top_k)
            continue
        score = 0
        for id in res.db_ids:
            try:
                ind = res.actual_ids.index(id)
                if ind > res.top_k * 3:
                    print('not in first section')
                    score -= ind
            except:
                print('id not exist')
                score -= len(res.actual_ids)
        scores.append(score)

    return sum(scores) / len(scores), sum(run_time) / len(run_time)

In [134]:
# indexing and searching if saved_db created

db = IvfDb(file_path="saved_db.bin")
db.build_index()
records_np = db.rertive_embeddings()

In [135]:
res = run_queries(db, records_np, 5, 10)
print(eval(res))
res = run_queries(db, records_np, 5, 10)
print(eval(res))
res = run_queries(db, records_np, 5, 10)
print(eval(res))
res = run_queries(db, records_np, 5, 10)
print(eval(res))
res = run_queries(db, records_np, 5, 10)
print(eval(res))

not in first section
not in first section
not in first section
not in first section
not in first section
not in first section
not in first section
not in first section
not in first section
not in first section
not in first section
not in first section
not in first section
not in first section
not in first section
not in first section
(-65.7, 10.21569983959198)
not in first section
not in first section
not in first section
not in first section
not in first section
not in first section
not in first section
not in first section
(-33.2, 12.408962416648865)
not in first section
not in first section
not in first section
not in first section
not in first section
not in first section
not in first section
not in first section
not in first section
not in first section
not in first section
not in first section
not in first section
not in first section
not in first section
(-2992.2, 8.245142960548401)
not in first section
not in first section
not in first section
not in first section
not in first 